In [4]:
# DQN
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gymnasium as gym

%matplotlib inline

In [5]:
'''
use the dense to generate the action
'''
class Network():
    def __init__(self, actionNum, stateShape, learningRate):

        self.model = keras.Sequential([
            keras.layers.Dense(32, input_shape = (stateShape, )),
            keras.layers.Dense(64),
            keras.layers.LeakyReLU(),
            keras.layers.Dense(actionNum),
        ])

        self.model.compile(
            loss = 'mse',
            optimizer = keras.optimizers.Adam(learningRate)
        )
        
    def Call(self, state):
        return self.model(state, training = False)

    def CopyVariable(self, network):
        '''
        param:
            network:    the other network model
                type:   Sequencial
        func:
            Copy the variable of other network model to self 
        '''
        for selfLayer, otherLayer in zip(self.model.layers, network.layers):
            selfLayer.set_weights(otherLayer.get_weights())

class DQN(keras.Model):
    def __init__(self, actionNum, stateShape, epsilon, batchSize, poolSize, learningRate, environment):
        '''
        param:
            actionNum:      the number of action
            stateShape:     the shape of state
            epsilon:        the parameter which is used in e-Greedy
            batchSize:      the batch size
            poolSize:       the size of memory pool
            learningRate:   for optimize the network
        '''
        super().__init__()
    
        self.environment = environment

        self.actionNum = actionNum
        self.stateShape = stateShape
        self.epsilon = epsilon
        self.batchSize = batchSize
        self.poolSize = poolSize

        self.chooseActionNet = Network(actionNum, stateShape, learningRate)
        self.targetNetwork = Network(actionNum, stateShape, learningRate)

        self.learningCount = 0

        self.memoryPool = np.zeros((self.poolSize, self.stateShape))


    def ChooseAction(self, state):
        if np.random.uniform() > self.epsilon:
            return np.random.randint(self.actionNum)

        predictValue = self.chooseActionNet.Call(state)
        return np.argmax(predictValue)

    def StoreMemory(self): ...

    def UpdateNetwork(self): ...

    def CopyNetwork(self): ...

    def Train(self):
        isDone = False
        state = self.environment.reset()

        while not isDone:
            action = self.ChooseAction(state)

            state, reward, isDone, _, _ = self.environment.step(action)

            
        


In [13]:
env = gym.make('CartPole-v1')
ACTIONNUM = env.action_space.n
STATESHAPE = env.observation_space.shape[0]
EPSILON = 0.7
BATCHSIZE = 64
POOLSIZE = 2000
LEARNINGRATE = 0.3